In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
from PIL import Image
from datetime import datetime
import os, json, cv2, random, pathlib, shutil
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
CATEGORIES = ["Open", "close", "Unknown"]
COLORS = {
    "Open": (255, 0, 0),  # Red
    "close": (0, 255, 0),  # Green
    "Unknown": (0, 0, 255)  # Blue
}

def filter_annotations(img_dir, label_filename):
    r""" Filter annotations by existing image files in `img_dir` folder
    
    Returns a dictionary with fields:
        "image": image file path
        "annotations": annotation JSON dict with Label studio format. E.g.,
            {
                "original_width": 1920,
                "original_height": 1280,
                "image_rotation": 0,
                "value": {
                    "x": 3.1,
                    "y": 8.2,
                    "radiusX": 20,
                    "radiusY": 16,
                    "ellipselabels": ["Car"]
                }
            }
    """
    orig_annotations = None
    with open(label_filename, "r") as f:
        orig_annotations = json.load(f)

    annotations = []
    for annotation in orig_annotations:
        # Process the label file, remove the first 9 random charactors 
        img_filename = os.path.basename(annotation["data"]["image"])[9:]
        
        # Check whether images exist with TIFF format
        filename, file_extension = os.path.splitext(img_filename)
        tif_filename = filename + '.tif'
        # if TIFF image file name starts with "2020", removes it
        if tif_filename.startswith("2020"):
            tif_filename = tif_filename[4:]
        tif_img_filepath = os.path.join(img_dir, tif_filename)
        
        if os.path.exists(tif_img_filepath):
            annotations.append({
                "image": tif_img_filepath,
                "annotations": annotation["annotations"][0]["result"]
            }) 
        else:
            print("Not exist: {}".format(tif_img_filepath))
    return annotations

def gen_ellipse_from_annotation(label):
    r"""
    Generate ellipse from Label Studio ellipse annotation
    Warning: this function only handles annotation with "image_rotation" = 0, otherwise, return False.

    Returns a tuple of
        - A bool flag to indicate whether the operation is successful
        - ellipse center (x, y)
        - ellipse axis (horizontal axis, vertical axis)
        - ellipse angle
        - category of the annotation

    """
    image_rotation = label["image_rotation"]
    if image_rotation != 0:
        return False, (0, 0), (0, 0), 0, ""
    img_w, img_h = label["original_width"], label["original_height"]
    rx, ry = label["value"]["radiusX"] * img_w / 100, label["value"]["radiusY"] * img_h / 100   # horizontal, verticle axies 
    # According to Label Studio, (x, y) coordinate of the top left corner before rotation (0, 100), but here it is actually the centre, weird.
    cx, cy = label["value"]["x"] * img_w / 100, label["value"]["y"] * img_h / 100
    angle = label["value"]["rotation"]  # clockwise degree
    category = label["value"]["ellipselabels"][0]
    return True, (cx, cy), (rx, ry), angle, category

def gen_polygon_from_annotation(label, delta=10):
    r""" 
    Generate polygon from Label Studio ellipse annotation
    Warning: this function only handles annotation with "image_rotation" = 0, otherwise, return False.

    Returns a tuple of
        - A bool flag to indicate whether the operation is successful
        - a closed polygon if successful, otherwise an empty list. E.g, [[x1, y1], [x2, y2], ...]
        - category of the annotation

    """
    success, center, axes, angle, category = gen_ellipse_from_annotation(label)
    if success:
        int_center = (int(center[0]), int(center[1]))
        int_axes = (int(axes[0]), int(axes[1]))
        int_angle = int(angle)
        poly = cv2.ellipse2Poly(center=int_center, axes=int_axes, angle=int_angle, arcStart=0, arcEnd=360, delta=delta)
        return True, poly, category
    else:
        return False, [], ""

def gen_polygon_w_boundingbox_from_annotation(label, delta=10):
    r"""
    Generate polygon and non-rotated bounding_box from Label Studio ellipse annotation
    Warning: this function only handles annotation with "image_rotation" = 0, otherwise, return False.

    Returns a tuple of
        - A bool flag to indicate whether the operation is successful
        - a closed polygon if successful, otherwise an empty list. E.g, [[x1, y1], [x2, y2], ...]
        - a bounding box in the format of (top_left_x, top_left_y, width, height)
        - category of the annotation

    """
    success, poly, category = gen_polygon_from_annotation(label, delta)
    if success:
        # bounding box format: (tlx, tly, w, h)
        bb = cv2.boundingRect(poly)
        return True, poly, bb, category
    else:
        return False, [], (0, 0, 0, 0), ""
    
def draw_annotations(img_filename, annotations, draw_polygon=True, draw_boundingbox=True, thickness=1):
    img = cv2.imread(img_filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img_h, img_w = img.shape[:2]
    # print("Image size = {}".format((img_h, img_w)))

    for v in annotations:
        if draw_boundingbox:
            success_bb, _, (x, y, w, h), category = gen_polygon_w_boundingbox_from_annotation(v, delta=10)
            if success_bb:
                cv2.rectangle(img, pt1=(x, y), pt2=(x+w, y+h), color=COLORS[category], thickness=thickness)
        if draw_polygon:
            success_poly, poly, category = gen_polygon_from_annotation(v)
            if success_poly:
                cv2.polylines(img, [poly], isClosed=True, thickness=thickness, color=COLORS[category])
                # cv2.fillConvexPoly(img, poly, color=COLORS[category])
    return img

def get_detectron2_dicts(img_dir, json_filename, delta=5):
    labelstudio_annotations = filter_annotations(img_dir, json_filename)

    dataset_dicts = []
    for idx, v in enumerate(labelstudio_annotations):
        record = {}
        img_filename = v["image"]
        annotations = v["annotations"]
        img_h, img_w = cv2.imread(img_filename).shape[:2]
        
        record["file_name"] = img_filename
        record["image_id"] = idx
        record["height"] = img_h
        record["width"] = img_w

        objs = []
        for anno in annotations:
            if anno["original_width"] != record["width"] or anno["original_height"] != record["height"]:
                print("Generate record error!")
                return []
            
            success, poly, _, category = gen_polygon_w_boundingbox_from_annotation(anno, delta=delta)
            # Convert from [[x1, y1], [x2, y2], ...] to [x1, y1, x2, y2, ...]
            px = [x for x, _ in poly]
            py = [y for _, y in poly]
            poly = [(float(x), float(y)) for x, y in poly]
            poly = [p for x in poly for p in x]
            if success:
                if len(poly) <= 4:
                    continue
                obj = {
                    "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": [poly],
                    "category_id": 0,  # single category
                }
                objs.append(obj)
            else:
                print("Generate record error!")
                return []
        record["annotations"] = objs
        dataset_dicts.append(record)
    
    return dataset_dicts

# Split data into train and val

Randomly split data into training (80%) and validation (20%) sets

In [ ]:
prj_path = pathlib.Path().absolute()
dataset_name = "stomata100"
img_dir = os.path.join(prj_path, "{}/images".format(dataset_name))
json_filename = os.path.join(prj_path, "{}/labels/labels.json".format(dataset_name))
labelstudio_annotations = filter_annotations(img_dir, json_filename)

# Random Shuffle
random.seed(0)
images = [v['image'] for v in labelstudio_annotations]
random.shuffle(images)

# Split
train_percentage = 0.8
train_num = int(len(images) * train_percentage)

# Training set
train_dir = os.path.join(prj_path, "{}/train".format(dataset_name))
os.makedirs(train_dir, exist_ok=True)
for v in images[:train_num]:
    dst = os.path.join(train_dir, os.path.basename(v))
    shutil.copyfile(v, dst)
    
# Validation set
val_dir = os.path.join(prj_path, "{}/val".format(dataset_name))
os.makedirs(val_dir, exist_ok=True)
for v in images[train_num:]:
    dst = os.path.join(val_dir, os.path.basename(v))
    shutil.copyfile(v, dst)

# Train on Stomata dataset
## Dataset visualization

To verify the dataset is in correct format, let's visualize the annotations of randomly selected samples in the training set:

In [ ]:
prj_path = pathlib.Path().absolute()
dataset_name = "stomata100"
img_dir = os.path.join(prj_path, "{}/images".format(dataset_name))
json_filename = os.path.join(prj_path, "{}/labels/labels.json".format(dataset_name))

# Get Detectron2 ground truth
for d in ["train", "val"]:
    catelog_name = "{}_{}".format(dataset_name, d)
    if catelog_name in DatasetCatalog:
        DatasetCatalog.remove(catelog_name)
    if catelog_name in MetadataCatalog:
        MetadataCatalog.remove(catelog_name)
    
    img_dir = os.path.join(prj_path, "{}/{}".format(dataset_name, d))
    DatasetCatalog.register(catelog_name, 
        lambda d=d: get_detectron2_dicts(os.path.join(prj_path, "{}/{}".format(dataset_name, d)), json_filename))
    MetadataCatalog.get(catelog_name).set(thing_classes=["stomata"])

stomata_metadata = MetadataCatalog.get("{}_train".format(dataset_name))

In [ ]:
prj_path = pathlib.Path().absolute()
dataset_name = "stomata100"
img_dir = os.path.join(prj_path, "{}/images".format(dataset_name))
json_filename = os.path.join(prj_path, "{}/labels/labels.json".format(dataset_name))

train_dataset_dicts = get_detectron2_dicts(os.path.join(prj_path, "{}/train".format(dataset_name)), json_filename, delta=5)

# Draw annotation ground truth
for idx, d in enumerate(train_dataset_dicts[:5]):
    img = cv2.imread(d["file_name"])       
    visualizer = Visualizer(img[:, :, ::-1], metadata=stomata_metadata, scale=1)
    out = visualizer.draw_dataset_dict(d)
    
    plt.figure()
    plt.imshow(out.get_image())
    plt.axis('off')
    plt.title("Ground truth: {}".format(os.path.basename(d["file_name"])))

# Train

In [ ]:
from detectron2.engine import DefaultTrainer

dataset_name = "stomata100"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("{}_train".format(dataset_name),)
cfg.DATASETS.TEST = ("{}_val".format(dataset_name), )
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 8000    # you may need to train longer for a practical dataset 300, 1000?
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (stomata). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.MODEL.DEVICE = "cpu"    # train with CPU

now = datetime.now()
dt_string = now.strftime("%Y_%m_%d_%H_%M_%S")
cfg.OUTPUT_DIR= "./{}_output_ep{}_{}".format(dataset_name, cfg.SOLVER.MAX_ITER, dt_string)

print(cfg)


In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

# Inference & evaluation using the trained model

Now, let's run inference with the trained model on the stomata validation dataset. First, let's create a predictor using the model we just trained.
Inference should use the config with parameters that are used in training
cfg now already contains everything we've set previously. We changed it a little bit for inference:

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.INPUT.MIN_SIZE_TEST = 0 # disable resize in testing
predictor = DefaultPredictor(cfg)

We can also evaluate its performance using AP metric implemented in COCO API.

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

dataset_name = "stomata100"

evaluator = COCOEvaluator("{}_val".format(dataset_name), output_dir="./output")
val_loader = build_detection_test_loader(cfg, "{}_val".format(dataset_name))
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`